# Thermodynamic formalism for breakome

For consistency with literature notation, cut points are assumed to be geneated by some discrete dynamics system $x_{n+1}=T(x_n)$ (but this assumption might not be necessary), and some phase space observable $u(x_n)=$umi number at $x_n$.

We consider local averages $U_n(x)=\Sigma_{k=0}^{n-1}\frac{u\left(x_k\right)}{n}$, and study their characteristic function
$$
\Phi(q) = \lim_{n\rightarrow \infty} \frac{1}{n} ln \langle \exp\left(qnU_n(x)\right) \rangle,
$$

where $\langle \cdot \rangle$ denotes the ensemble average, which is assumed to be equivalent (or at least well approximated by) the large space\time average (self-averaging):
$$
\langle g(x) \rangle = \lim_{N\rightarrow \infty} \frac{1}{N} \Sigma_{k=0}^{N-1} g\left(x_k\right).
$$

In [43]:
import pandas as pd
import numpy as np
import math
import scipy.sparse as sparse
import pickle
import scipy.io
import subprocess
#################################################
def phi(q,n,uu):
    phi = q*n*uu
    phi = phi.expm1()+np.ones(phi.shape)
    phi = phi.mean()
    phi = np.log(phi)/n
    return phi

def expandlist(i,n):
    ii = list(i)
    for point in i:
        ii.extend(range(point-n, point))
    ii = list(set(ii))
    ii.sort()
    return ii
#################################################
n = 100
numproc = 8
#################################################
filename = 'chr.loc.uminumb'
data = pd.read_csv(filename, sep=' ',header=None, index_col=False)
#################################################
chromosomes = ['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8',
               'chr9','chr10','chr11','chr12','chr13','chr14','chr15',
               'chr16','chr17','chr18','chr19','chr20','chr21','chr22',
               'chrX','chrY']

chr_size={"chr1":249250621,"chr2":243199373,"chr3":198022430,"chr4":191154276,"chr5":180915260,"chr6":171115067,
          "chr7":159138663,"chr8":146364022,"chr9":141213431,"chr10":135534747,"chr11":135006516,
          "chr12":133851895,"chr13":115169878,"chr14":107349540,"chr15":102531392,"chr16":90354753,
          "chr17":81195210,"chr18":78077248,"chr19":59128983,"chr20":63025520,"chr21":48129895,
          "chr22":51304566,"chrX":155270560,"chrY":59373566}
#################################################
if data.shape[1]==3:
    data.columns = ['CHR','LOC','UMI']

chromosome = "chr1"
chrdata = data[data.CHR == chromosome]
#################################################
i = chrdata['LOC'].tolist()
j = [0]*(len(i))
uu = chrdata['UMI'].tolist()
M = chr_size['chr1']
N = 1

u = sparse.csr_matrix((uu,(i,j)),shape=(M,N))

ii = expandlist(i,n)

sub_ii = [ii[i::numproc] for i in range(numproc)]

ind = 1
for seq in sub_ii:
    name = 'seq_' + str(ind) 
    outfile = open(name, "w")
    pickle.dump(seq, outfile)
    outfile.close()
    ind = ind+1
#################################################
scipy.io.mmwrite('u_vector',u)
    
outfile = open('numproc_file', "w")
pickle.dump(numproc, outfile)
outfile.close()

outfile = open('n_file', "w")
pickle.dump(n, outfile)
outfile.close()
#################################################
print "Start parallel execution.."
subprocess.call("./parallel_script.sh")
print "Done"
#################################################
lista = []
for ind in range(1,numproc+1):
    infile = open('out_seq_'+str(ind), "r")
    lista.extend(pickle.load(infile))

df = pd.DataFrame(lista)
df = df.sort(columns=0)

df.columns = ['LOC','U']
x = df['LOC'].tolist()
y = [0]*(len(x))
uu = df['U'].tolist()
M = chr_size[chromosome]
N = 1

uu_sparse = sparse.csr_matrix((uu,(x,y)),shape=(M,N))
#################################################
q = -0.2
phi(q,n,uu_sparse)

The bottleneck is in the evaluation of u(x,n) for all x's.
I need to create a new sparse vector, which has non-zero entries in all left half-balls of radius n centered in x. 
For those locations I need to evalute the interval average. 
This takes approx 0.5 sec per 1000 points.
If n, the window size, is 10000 then we have to evaluate 20M points...meaning 20K secs...
I need to parallelize the evaluation of U(x,n,u) over x. Using 32 processors it should be possible to run it in 600 secs.

expandlist() expand the input list i to include windows of size n, to the left, which will become non-zero values for the n-local fields evaluated as n-local averages.

In order to evaluate the fiels U(x,n,u) at all x, one first set a sparse 0 vector U(). The non-zero entries will be obtained by evaluating U at the points in ii. ii is still too large, so one can parallelize the calculation dividing ii into the max number of processors available. 

You need to store u locally, as well as numbproc and n.

When this is done you need to call the parallel_script.sh, which distributed local_field.py over the n processors.

The output files are stored locally in files named out\_seq\_[1...numproc]. 
They have to be loaded, merged and sorted according to their x coordinate.

Then convert the local field's values to a sparse vector 

Now you need to evaluate the average [exp(q*n*uu_sparse(x))]\_x; which is a sparse array of ones and other non-one values. After constructing this sparse vector you have to average it, and take the logarithm of the outcome in order to have the characteristic function:
$$
\Phi(q) = \lim_{n\rightarrow \infty} \frac{1}{n} ln \langle \exp\left(qnU_n(x)\right) \rangle_x,
$$


You need to plot phi(q,n,u) as a function of n, and find the n for which phi does not change anymore. In this regime, for n sufficiently large, you can infer properties of phi which do not depend on finite size effect. 